In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import *
from sklearn.feature_selection import *

from imblearn.under_sampling import RandomUnderSampler


In [2]:
shop_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/data/shopper_data_slm_feat.csv')
shop_df.drop(['session_id_hash', 'product_action', 'reduced_time', 'HVGms_edges', 'pattern_hvg_4_nodes', 'pattern_hvg_5_nodes', 'unigram', 'bigram', 'trigram', 'unigram_prob_freq', 'bigram_prob_freq', 'trigram_prob_freq', 'pattern_hvg_4_nodes_prob_freq', 'pattern_hvg_5_nodes_prob_freq'], axis=1, inplace=True)
shop_df['conversion_class'] = np.where(shop_df['conversion_class'] == 'NC', 0, 1)
print(shop_df.shape)
shop_df.head()

(10392, 140)


,conversion_class,unigram_entropy,bigram_entropy,trigram_entropy,pattern_hvg_4_nodes_entropy,pattern_hvg_5_node_entropy,"(1,)","(2,)","(6,)","(3,)",...,S5,R5,K5,C5,D5,H5,J5,M5,F5,I5
0,0,0.601924,0.601924,1.450805,1.054920,1.332179,0.812500,0.125000,0.0625,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.661563,0.661563,1.011404,0.636514,1.098612,0.625000,0.375000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.679193,0.679193,1.193550,0.636514,1.098612,0.583333,0.416667,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.673012,0.673012,0.974315,1.332179,1.609438,0.600000,0.400000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.678209,0.678209,1.625267,1.265001,1.704551,0.413793,0.586207,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
x, y = shop_df.drop('conversion_class', axis=1), shop_df['conversion_class']
rus = RandomUnderSampler()
subsamples = []
for _ in range(10):
    X_resampled, y_resampled = rus.fit_resample(x, y)
    subsamples.append((X_resampled, y_resampled))
    

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(8313, 139) (2079, 139) (8313,) (2079,)


# MI (Mutual Information)

In [27]:
mi_feat_list_10_percentile = []
mi_feat_list_20_percentile = []
mi_feat_list_30_percentile = []
mi_feat_list_50_percentile = []

for samples in subsamples:
    x_sample = samples[0]
    y_sample = samples[1]
    
    sel_10_percentile_cols = SelectPercentile(mutual_info_classif, percentile=10)
    sel_10_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_10_percentile.append(x_sample.columns[sel_10_percentile_cols.get_support()])
    
    sel_20_percentile_cols = SelectPercentile(mutual_info_classif, percentile=20)
    sel_20_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_20_percentile.append(x_sample.columns[sel_20_percentile_cols.get_support()])
    
    sel_30_percentile_cols = SelectPercentile(mutual_info_classif, percentile=30)
    sel_30_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_30_percentile.append(x_sample.columns[sel_30_percentile_cols.get_support()])
    
    sel_50_percentile_cols = SelectPercentile(mutual_info_classif, percentile=50)
    sel_50_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_50_percentile.append(x_sample.columns[sel_50_percentile_cols.get_support()])

print(len(mi_feat_list_10_percentile[0]))
print(len(mi_feat_list_20_percentile[0]))
print(len(mi_feat_list_30_percentile[0]))
print(len(mi_feat_list_50_percentile[0]))

14
28
42
69


# mRMR

In [50]:
mrmr_feat_list_10_percentile = []
mrmr_feat_list_20_percentile = []
mrmr_feat_list_30_percentile = []
mrmr_feat_list_50_percentile = []


for samples in subsamples:
    x_sample = samples[0]
    y_sample = samples[1]
    
    F = pd.Series(f_regression(x_sample, y_sample)[0], index = x_sample.columns)
    corr = pd.DataFrame(.001, index = x_sample.columns, columns = x_sample.columns)


    # initialize list of selected features and list of excluded features
    selected = []
    not_selected = x_sample.columns.to_list()
    k = 14
    # repeat k times
    for i in range(k):
    
        if i > 0:
            last_selected = selected[-1]
            corr.loc[not_selected, last_selected] = x_sample[not_selected].corrwith(x_sample[last_selected]).abs().clip(.00001)
            
        score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
        best = score.index[score.argmax()]
        selected.append(best)
        not_selected.remove(best)
    mrmr_feat_list_10_percentile.append(selected)
    
    selected = []
    not_selected = x_sample.columns.to_list()
    k = 28
    # repeat k times
    for i in range(k):
    
        if i > 0:
            last_selected = selected[-1]
            corr.loc[not_selected, last_selected] = x_sample[not_selected].corrwith(x_sample[last_selected]).abs().clip(.00001)
            
        score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
        best = score.index[score.argmax()]
        selected.append(best)
        not_selected.remove(best)
    mrmr_feat_list_20_percentile.append(selected)
     
    selected = []
    not_selected = x_sample.columns.to_list()   
    k = 42
    # repeat k times
    for i in range(k):
    
        if i > 0:
            last_selected = selected[-1]
            corr.loc[not_selected, last_selected] = x_sample[not_selected].corrwith(x_sample[last_selected]).abs().clip(.00001)
            
        score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
        best = score.index[score.argmax()]
        selected.append(best)
        not_selected.remove(best)
    mrmr_feat_list_30_percentile.append(selected)
      
    selected = []
    not_selected = x_sample.columns.to_list()  
    k = 60
    # repeat k times
    for i in range(k):
    
        if i > 0:
            last_selected = selected[-1]
            corr.loc[not_selected, last_selected] = x_sample[not_selected].corrwith(x_sample[last_selected]).abs().clip(.00001)
            
        score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
        best = score.index[score.argmax()]
        selected.append(best)
        not_selected.remove(best)
    mrmr_feat_list_50_percentile.append(selected)
        
        
print(len(mrmr_feat_list_10_percentile[0]))
print(len(mrmr_feat_list_20_percentile[0]))
print(len(mrmr_feat_list_30_percentile[0]))
print(len(mrmr_feat_list_50_percentile[0]))

14
28
42
60


# MI & mRMR

In [62]:
mi_mrmr_feat_list_10_percentile = []
for feat in zip(mi_feat_list_10_percentile, mrmr_feat_list_10_percentile):
    mi_mrmr_feat_list_10_percentile.append(list(set(feat[0]).intersection(feat[1])))

mi_mrmr_feat_list_20_percentile = []
for feat in zip(mi_feat_list_20_percentile, mrmr_feat_list_20_percentile):
    mi_mrmr_feat_list_20_percentile.append(list(set(feat[0]).intersection(feat[1])))
    
mi_mrmr_feat_list_30_percentile = []
for feat in zip(mi_feat_list_30_percentile, mrmr_feat_list_30_percentile):
    mi_mrmr_feat_list_30_percentile.append(list(set(feat[0]).intersection(feat[1])))
    
mi_mrmr_feat_list_50_percentile = []
for feat in zip(mi_feat_list_50_percentile, mrmr_feat_list_50_percentile):
    mi_mrmr_feat_list_50_percentile.append(list(set(feat[0]).intersection(feat[1]))) 


# Saving in CSV and Text Files

In [63]:
i = 1
for samples in subsamples:
    sample_df = pd.concat([samples[0], samples[1]], axis = 1)
    sample_df.to_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/subsamples/subsample_' + str(i) + '.csv')
    i+=1

In [78]:
def write_list_of_lists_to_file(list_of_lists, file_path):
    with open(file_path, 'w') as file:
        for sublist in list_of_lists:
            line = ' '.join(str(element) for element in sublist) + '\n'
            file.write(line)

In [80]:
write_list_of_lists_to_file(mi_feat_list_10_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/features/mi_feat_list_10_percentile.txt')
write_list_of_lists_to_file(mi_feat_list_20_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/features/mi_feat_list_20_percentile.txt')
write_list_of_lists_to_file(mi_feat_list_30_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/features/mi_feat_list_30_percentile.txt')
write_list_of_lists_to_file(mi_feat_list_50_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/features/mi_feat_list_50_percentile.txt')

write_list_of_lists_to_file(mrmr_feat_list_10_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/features/mrmr_feat_list_10_percentile.txt')
write_list_of_lists_to_file(mrmr_feat_list_20_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/features/mrmr_feat_list_20_percentile.txt')
write_list_of_lists_to_file(mrmr_feat_list_30_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/features/mrmr_feat_list_30_percentile.txt')
write_list_of_lists_to_file(mrmr_feat_list_50_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/features/mrmr_feat_list_50_percentile.txt')

write_list_of_lists_to_file(mi_mrmr_feat_list_10_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/features/mi_mrmr_feat_list_10_percentile.txt')
write_list_of_lists_to_file(mi_mrmr_feat_list_20_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/features/mi_mrmr_feat_list_20_percentile.txt')
write_list_of_lists_to_file(mi_mrmr_feat_list_30_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/features/mi_mrmr_feat_list_30_percentile.txt')
write_list_of_lists_to_file(mi_mrmr_feat_list_50_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/features/mi_mrmr_feat_list_50_percentile.txt')


In [75]:
len(mi_feat_list_10_percentile)

10

pandas.core.indexes.base.Index

In [69]:
len(mi_feat_list_10_percentile)

10

In [29]:

# https://towardsdatascience.com/mrmr-explained-exactly-how-you-wished-someone-explained-to-you-9cf4ed27458b

['(3, 1, 1)', '(2, 4)', 'K5', 'M5', '(3, 1, 4)']
